# AML HW3 Problem 1

In [ ]:
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [ ]:
# (b)

# Load training data
train_labels, train_data = [], []

with open('./faces/train.txt', 'r') as f:
    for line in f:
        path, label = line.strip().split()
        img = imageio.imread(path)
        train_data.append(img.reshape(2500,))   # 50 x 50 = 2500
        train_labels.append(int(label))

train_data = np.array(train_data, dtype=float)
train_labels = np.array(train_labels, dtype=int)

print("Train data shape:", train_data.shape)
print("Train labels shape:", train_labels.shape)

plt.imshow(train_data[10, :].reshape(50, 50), cmap=cm.Greys_r)
plt.title(f"Train sample, label={train_labels[10]}")
plt.axis('off')
plt.show()

In [ ]:
# Load test data
test_labels, test_data = [], []

with open('./faces/test.txt', 'r') as f:
    for line in f:
        path, label = line.strip().split()
        img = imageio.imread(path)
        test_data.append(img.reshape(2500,))
        test_labels.append(int(label))

test_data = np.array(test_data, dtype=float)
test_labels = np.array(test_labels, dtype=int)

print("Test data shape:", test_data.shape)
print("Test labels shape:", test_labels.shape)

plt.imshow(test_data[10, :].reshape(50, 50), cmap=cm.Greys_r)
plt.title(f"Test sample, label={test_labels[10]}")
plt.axis('off')
plt.show()

In [ ]:
# (c)

# Compute average face
mu = np.mean(train_data, axis=0)
print("Average face vector shape:", mu.shape)

plt.imshow(mu.reshape(50, 50), cmap=cm.Greys_r)
plt.title("Average Face (μ)")
plt.axis('off')
plt.show()

In [ ]:
# (d)

# Mean subtraction for training data
X_centered = train_data - mu

plt.imshow(X_centered[10, :].reshape(50, 50), cmap=cm.Greys_r)
plt.title("Mean-Subtracted Training Face")
plt.axis('off')
plt.show()

# Mean subtraction for test data
Xtest_centered = test_data - mu

plt.imshow(Xtest_centered[10, :].reshape(50, 50), cmap=cm.Greys_r)
plt.title("Mean-Subtracted Test Face")
plt.axis('off')
plt.show()

In [ ]:
# (e)

# Compute covariance matrix C = X^T X (2500 x 2500)
C = X_centered.T @ X_centered

# Eigendecomposition
eigvals, eigvecs = np.linalg.eigh(C)

# Sort eigenpairs
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

# Normalize eigenvectors
for i in range(eigvecs.shape[1]):
    norm = np.linalg.norm(eigvecs[:, i])
    if norm > 0:
        eigvecs[:, i] = eigvecs[:, i] / norm

# Select top 10 eigenfaces
top_k = 10
eigenfaces = eigvecs[:, :top_k].T   # shape (top_k, 2500) where row i is i-th eigenface

# Display
plt.figure(figsize=(14, 5))
for i in range(top_k):
    ef = eigenfaces[i]
    ef_disp = (ef - ef.min()) / (ef.max() - ef.min() + 1e-12)
    ax = plt.subplot(2, 5, i+1)
    plt.imshow(ef_disp.reshape(50, 50), cmap=cm.Greys_r)
    plt.title(f"Eigenface {i+1}")
    plt.axis('off')
plt.suptitle("Top 10 Eigenfaces (from largest eigenvalues)")
plt.tight_layout()
plt.show()

In [ ]:
# (f)

def compute_eigenface_features(X_centered, Xtest_centered, eigvecs, r):
    Vr = eigvecs[:, :r]          # select top-r eigenfaces
    F = X_centered @ Vr          # training features
    F_test = Xtest_centered @ Vr # test features
    return F, F_test

In [ ]:
# (g)

def train_and_eval_logreg(F_train, y_train, F_test, y_test, solver='saga', max_iter=2000, C=1.0, n_jobs=-1):
    base_clf = LogisticRegression(solver=solver, max_iter=max_iter, C=C, tol=1e-4)
    clf = OneVsRestClassifier(base_clf, n_jobs=n_jobs)
    pipe = make_pipeline(StandardScaler(), clf)

    pipe.fit(F_train, y_train)
    y_pred = pipe.predict(F_test)
    acc = accuracy_score(y_test, y_pred)
    return acc, pipe

r = 10
F, F_test = compute_eigenface_features(X_centered, Xtest_centered, eigvecs, r)

acc, model = train_and_eval_logreg(F, train_labels, F_test, test_labels, solver='saga', max_iter=2000, C=1.0, n_jobs=-1)
print(f"Accuracy with r={r}: {acc*100:.2f}%")

In [ ]:
accuracies = []
r_values = range(1, 201)
for r in tqdm(r_values, desc="Training Logistic Regression for r values"):
    F, F_test = compute_eigenface_features(X_centered, Xtest_centered, eigvecs, r)
    acc, _ = train_and_eval_logreg(F, train_labels, F_test, test_labels,
                                   solver='saga', max_iter=2000, C=1.0, n_jobs=-1)
    accuracies.append(acc)

plt.figure(figsize=(8, 5))
plt.plot(r_values, np.array(accuracies)*100, marker='o', linewidth=1)
plt.xlabel("Number of Eigenfaces (r)")
plt.ylabel("Test Classification Accuracy (%)")
plt.title("Eigenface Logistic Regression (One-vs-Rest)")
plt.grid(True)
plt.show()

# best result
best_r = r_values[np.argmax(accuracies)]
best_acc = max(accuracies)
print(f"Best accuracy = {best_acc*100:.2f}% at r = {best_r}")

In [ ]:
# (h)

n_train = train_data.shape[0]
max_r = 200
ds = np.zeros(max_r)

for r in range(1, max_r + 1):
    Vr = eigvecs[:, :r]
    F_r = X_centered @ Vr       # Project
    Xrec_centered = F_r @ Vr.T  # Reconstruct centered
    Xrec = Xrec_centered + mu   # Add mean back
    diff = train_data - Xrec    # Compute Frobenius norm
    frob = np.linalg.norm(diff, ord='fro')
    ds[r-1] = frob

plt.figure(figsize=(8,5))
plt.plot(np.arange(1, max_r+1), ds, marker='o', markersize=3)
plt.xlabel('Number of eigenfaces r')
plt.ylabel(r'Frobenius distance $d_r = \|X - X\'\|_F$')
plt.title('Low-rank approximation error vs r (Frobenius norm)')
plt.grid(True)
plt.tight_layout()
plt.show()

for r_check in [1, 5, 10, 20, 50, 100, 200]:
    print(f"r={r_check:3d}  Frobenius d = {ds[r_check-1]:.4f}")